**Download, unzip and convert 3DShapeNets/volumetric_data to train.tar and test.tar files (friendly format)**

In [0]:
!wget http://3dshapenets.cs.princeton.edu/3DShapeNetsCode.zip
!unzip 3DShapeNetsCode.zip

--2019-12-02 20:31:06--  http://3dshapenets.cs.princeton.edu/3DShapeNetsCode.zip
Resolving 3dshapenets.cs.princeton.edu (3dshapenets.cs.princeton.edu)... 128.112.136.51
Connecting to 3dshapenets.cs.princeton.edu (3dshapenets.cs.princeton.edu)|128.112.136.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 533450062 (509M) [application/zip]
Saving to: ‘3DShapeNetsCode.zip.1’

3DShapeNetsCode.zip 100%[===================>] 508.74M  41.8MB/s    in 13s     

2019-12-02 20:31:19 (40.4 MB/s) - ‘3DShapeNetsCode.zip.1’ saved [533450062/533450062]

Archive:  3DShapeNetsCode.zip
replace 3DShapeNets/kFunctions2.cu? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
!python --version

Python 3.6.8


In [0]:
class_id_to_name = {
    "1": "bathtub",
    "2": "bed",
    "3": "chair",
    "4": "desk",
    "5": "dresser",
    "6": "monitor",
    "7": "night_stand",
    "8": "sofa",
    "9": "table",
    "10": "toilet"
}
class_name_to_id = { v : k for k, v in class_id_to_name.items() }

class_names = set(class_id_to_name.values())

#from io import BytesIO
import io
import tarfile
import time
import zlib

import numpy as np

PREFIX = 'data/'
SUFFIX = '.npy.z'

class NpyTarWriter(object):
    def __init__(self, fname):
        self.tfile = tarfile.open(fname, 'w|')

    def add(self, arr, name):

        sio = BytesIO()
        np.save(sio, arr)
        zbuf = zlib.compress(sio.getvalue())
        sio.close()

        zsio = BytesIO(zbuf)
        tinfo = tarfile.TarInfo('{}{}{}'.format(PREFIX, name, SUFFIX))
        tinfo.size = len(zbuf)
        tinfo.mtime = time.time()
        zsio.seek(0)
        self.tfile.addfile(tinfo, zsio)
        zsio.close()

    def close(self):
        self.tfile.close()


class NpyTarReader(object):
    def __init__(self, fname):
        self.tfile = tarfile.open(fname, 'r|')

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def next(self):
        entry = self.tfile.next()
        if entry is None:
            raise StopIteration()
        name = entry.name[len(PREFIX):-len(SUFFIX)]
        fileobj = self.tfile.extractfile(entry)
        buf = zlib.decompress(fileobj.read())
        arr = np.load(BytesIO(buf))
        return arr, name

    def close(self):
        self.tfile.close()

In [0]:
!pip install Path.py

In [0]:
import logging
import random
import numpy as np
import scipy.io
from path import Path


def write(records, fname):
    writer = NpyTarWriter(fname)
    for (classname, instance, rot, fname) in records:
        class_id = int(class_name_to_id[classname])
        name = '{:03d}.{}.{:03d}'.format(class_id, instance, rot)
        arr = scipy.io.loadmat(fname)['instance'].astype(np.uint8)
        arrpad = np.zeros((32,)*3, dtype=np.uint8)
        arrpad[1:-1,1:-1,1:-1] = arr
        writer.add(arrpad, name)
    writer.close()

data_dir = Path('3DShapeNets')


logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s| %(message)s')

#base_dir = Path('~/code/3DShapeNets2/3DShapeNets/volumetric_data').expand()
base_dir = (data_dir/'volumetric_data').expand()

records = {'train': [], 'test': []}

logging.info('Loading .mat files')
for fname in sorted(base_dir.walkfiles('*.mat')):
    if fname.endswith('test_feature.mat') or fname.endswith('train_feature.mat'):
        continue
    elts = fname.splitall()
    instance_rot = Path(elts[-1]).stripext()
    instance = instance_rot[:instance_rot.rfind('_')]
    rot = int(instance_rot[instance_rot.rfind('_')+1:])
    split = elts[-2]
    classname = elts[-4].strip()
    if classname not in class_names:
        continue
    records[split].append((classname, instance, rot, fname))


# just shuffle train set
logging.info('Saving train npy tar file')
train_records = records['train']
random.shuffle(train_records)
write(train_records, 'shapenet10_train.tar')

# order test set by instance and orientation
logging.info('Saving test npy tar file')
test_records = records['test']
test_records = sorted(test_records, key=lambda x: x[2])
test_records = sorted(test_records, key=lambda x: x[1])
write(test_records, 'shapenet10_test.tar')


2019-12-02 23:52:11,630 INFO| Loading .mat files
2019-12-02 23:52:16,097 INFO| Saving train npy tar file
2019-12-02 23:52:52,564 INFO| Saving test npy tar file


# Set input for the CNN from the .tar files

In [0]:
def data_loader(fname):

    dims = (32, 32, 32)
    chunk_size = 12
    xc = np.zeros((12, 1,)+dims, dtype=np.float32)
#    xf = np.zeros((chunk_size*22+800, 1,)+dims, dtype=np.float32)
#    reader = NpyTarReader(fname)
    yc = []
    while True:
        reader = NpyTarReader(fname)
        for ix, (x, name) in enumerate(reader):
            cix = ix % chunk_size
            xc[cix] = x.astype(np.float32)
  #          xf[ix] = x.astype(np.float32)
            yc.append(int(name.split('.')[0])-1)
            if len(yc) == chunk_size:
                xc = jitter_chunk(xc)
                yield (2.0*xc - 1.0, np.asarray(yc, dtype=np.float32))
  #              xf[0:1] = (0, 1)
                yc = []
                xc.fill(0)
  #      if len(yc) > 0:
   #         # pad to nearest multiple of batch_size
    #        if len(yc)%32 != 0:
      #          new_size = int(np.ceil(len(yc)/float(32)))*32
     #           xc = xc[:new_size]
       #         xc[len(yc):] = xc[:(new_size-len(yc))]
        #        yc = yc + yc[:(new_size-len(yc))]

         #   xc = jitter_chunk(xc)
          #  yield (2.0*xc - 1.0, np.asarray(yc, dtype=np.float32))

def jitter_chunk(src):
    dst = src.copy()
    if np.random.binomial(1, .2):
        dst[:, :, ::-1, :, :] = dst
    if np.random.binomial(1, .2):
        dst[:, :, :, ::-1, :] = dst
    max_ij = 2
    max_k = 2
    shift_ijk = [np.random.random_integers(-max_ij, max_ij),
                 np.random.random_integers(-max_ij, max_ij),
                 np.random.random_integers(-max_k, max_k)]
    for axis, shift in enumerate(shift_ijk):
        if shift != 0:
            # beware wraparound
            dst = np.roll(dst, shift, axis+2)
    return dst

In [0]:
loader = data_loader('shapenet10_train.tar')
#xf = np.zeros((chunk_size*22+800, 1,)+dims, dtype=np.float32)
#yf = []
#for ix, (x_shared, y_shared) in loader:

    


# Imports 

In [0]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf

# The Model 

In [0]:
from keras import backend as K
from keras import optimizers
K.set_image_data_format('channels_first')

def my_init(shape, dtype=None):
        receptive_field_size = np.prod(shape[2:])
        c = shape[1] # input channels
        nl = c*receptive_field_size
        std = np.sqrt(2.0/(nl))
        return np.random.normal(0, std, size=shape)

# Define model
model = keras.Sequential()
model.add(layers.InputLayer(input_shape=(1, 32, 32, 32)))

model.add(layers.Conv3D(filters=32, kernel_size=(5,5,5), strides=(2,2,2),data_format='channels_first', kernel_initializer=my_init))
model.add(layers.LeakyReLU(alpha=0.1))

model.add(layers.Dropout(rate=0.2))

model.add(layers.Conv3D(filters=32, kernel_size=(3,3,3),data_format='channels_first', kernel_initializer=my_init))
model.add(layers.LeakyReLU(alpha=0.1))

model.add(layers.MaxPool3D(pool_size=(2,2,2), data_format='channels_first'))

model.add(layers.Dropout(rate=0.3))

model.add(layers.Flatten(data_format='channels_first'))

model.add(layers.Dense(units=128,kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)))

model.add(layers.Dropout(rate=0.4))

model.add(layers.Dense(units=10,kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)))

#model.add(layers.Convolution2D(16, (3, 3),
#                        padding='same',
#                        input_shape=x_train.shape[1:], activation='relu'))
#model.add(layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
#model.add(layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
#model.add(layers.MaxPooling2D(pool_size =(2,2)))
#model.add(layers.Flatten())
#model.add(layers.Dense(128, activation='relu'))
#model.add(layers.Dense(100, activation='softmax')) 
# Train model

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['mse', 'accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_18 (Conv3D)           (None, 32, 14, 14, 14)    4032      
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 32, 14, 14, 14)    0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 32, 14, 14, 14)    0         
_________________________________________________________________
conv3d_19 (Conv3D)           (None, 32, 12, 12, 12)    27680     
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 32, 12, 12, 12)    0         
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 32, 6, 6, 6)       0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 32, 6, 6, 6)     

# Training 

In [0]:
model.fit_generator(data_loader('shapenet10_train.tar'), steps_per_epoch=1430, verbose=2, epochs=50000)

Epoch 1/50000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: This function is deprecated. Please call randint(-2, 2 + 1) instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: This function is deprecated. Please call randint(-2, 2 + 1) instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: This function is deprecated. Please call randint(-2, 2 + 1) instead


1430/1430 - 15s - loss: 2.9665 - mean_squared_error: 88.2596 - acc: 0.1132
Epoch 2/50000
1430/1430 - 15s - loss: 3.0163 - mean_squared_error: 87.8953 - acc: 0.1189
Epoch 3/50000
1430/1430 - 16s - loss: 3.0313 - mean_squared_error: 86.7733 - acc: 0.1138
Epoch 4/50000
1430/1430 - 16s - loss: 2.9637 - mean_squared_error: 86.7553 - acc: 0.1153
Epoch 5/50000
1430/1430 - 16s - loss: 3.0369 - mean_squared_error: 86.9185 - acc: 0.1195
Epoch 6/50000
1430/1430 - 15s - loss: 3.0106 - mean_squared_error: 86.4720 - acc: 0.1138
Epoch 7/50000
1430/1430 - 15s - loss: 2.9740 - mean_squared_error: 87.1360 - acc: 0.1146
Epoch 8/50000
1430/1430 - 15s - loss: 3.0229 - mean_squared_error: 86.5876 - acc: 0.1216
Epoch 9/50000
1430/1430 - 15s - loss: 2.9890 - mean_squared_error: 86.9121 - acc: 0.1139
Epoch 10/50000
1430/1430 - 15s - loss: 2.9909 - mean_squared_error: 86.0723 - acc: 0.1177
Epoch 11/50000
1430/1430 - 15s - loss: 3.0539 - mean_squared_error: 85.6786 - acc: 0.1172
Epoch 12/50000
1430/1430 - 15s - 

# Testing 

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

Test accuarcy: 3348.04%


# Inference 

In [1]:
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline  
idx = randint(0, len(x_test))
img = x_test[idx]
plt.imshow(img.squeeze()) 
pred = model.predict(np.expand_dims(img, axis=0))[0]
ind = (-pred).argsort()[:5]
latex = [class_names[x] for x in ind]
print(latex)

NameError: ignored

# Store the classes 

In [0]:
with open('class_names.txt', 'w') as file_handler:
    for item in class_names:
        file_handler.write("{}\n".format(item))

# Install TensorFlowJS

In [0]:
!pip install tensorflowjs 

    100% |████████████████████████████████| 327kB 6.4MB/s 
    100% |████████████████████████████████| 12.2MB 3.5MB/s 
  Found existing installation: numpy 1.14.3
    Uninstalling numpy-1.14.3:
      Successfully uninstalled numpy-1.14.3
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6


# Save and Convert 

In [0]:
model.save('keras.h5')

In [0]:
!mkdir model
!tensorflowjs_converter --input_format keras keras.h5 model/

mkdir: cannot create directory ‘model’: File exists
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Zip and Download 

In [0]:
!cp class_names.txt model/class_names.txt

In [0]:
!zip -r model.zip model 

  adding: model/ (stored 0%)
  adding: model/group5-shard1of1 (deflated 7%)
  adding: model/model.json (deflated 82%)
  adding: model/group2-shard1of1 (deflated 7%)
  adding: model/group3-shard1of1 (deflated 7%)
  adding: model/class_names.txt (deflated 41%)
  adding: model/group1-shard1of1 (stored 0%)
  adding: model/group4-shard1of1 (deflated 7%)


In [0]:
from google.colab import files
files.download('model.zip')